In [28]:
import cv2 as cv
import numpy as np
import datetime as dtime
video_file = 'rtsp://210.99.70.120:1935/live/cctv002.stream'
target_format = 'avi'
target_fourcc = 'XVID'

video = cv.VideoCapture(video_file)

if(video.isOpened()):
    target = cv.VideoWriter()
    
    record_mode = False
    fps = int(video.get(cv.CAP_PROP_FPS)/1500) # video.get(cv.CAP_PROP_FPS) == 90000
    wait_msec = int(1/fps*1000+1) # +1 to avoid hesitation from flooring fps
    
    while True:
        valid, img  = video.read()
        if not valid:
            break
            
       
        if not target.isOpened() and record_mode:
            target_file = d.strftime('%Y%m%d')+'_'+d.strftime('%H%M%S')+'.' + target_format
            h, w, *_ = img.shape
            is_color = (img.ndim > 2) and (img.shape[2] > 1)
            target.open(target_file, cv.VideoWriter_fourcc(*target_fourcc), fps, (w, h), is_color)
       
        if record_mode:
            target.write(img)
            cv.putText(img, 'Recording', (10, 25), cv.FONT_HERSHEY_DUPLEX, 1, (66, 66, 255), thickness=2)

        cv.putText(img, f'{fps}fps', (560, 25), cv.FONT_HERSHEY_DUPLEX, 0.5, (127, 255, 127), thickness=1)
        cv.imshow('Video Player', img)

        key = cv.waitKey(wait_msec)
        if key == 27:
            break
        elif key == ord(' '):
            if record_mode:
                record_mode = False
                target.release()
                target = cv.VideoWriter()
            else:
                record_mode = True
                d = dtime.datetime.now()
        elif key == ord('<') or key == ord(','):
            fps = max(15,int(fps/2))
            wait_msec = int(1/fps*1000+1)
        elif key == ord('>') or key == ord('.'):
            fps = min(480,int(fps*2))
            wait_msec = int(1/fps*1000+1)

    cv.destroyAllWindows()